### Imports

In [ ]:
import scipy.io as scio
import numpy as np    
import matplotlib.pyplot as plt
import sys
import os
import math
import pprint
import cv2
from scipy.misc import imsave
from helper import *
from create_labels import *
from stats_helper import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'

In [ ]:
import skimage.restoration as sr
import numpy as np
import glob
import os
import scipy.io as scio
from skimage import exposure
from skimage.io import imsave, imread
from scipy.misc import imresize
from scipy.io import savemat
from scipy import ndimage, misc
import matplotlib.pyplot as plt
%matplotlib inline
import re

from helper import *
from stats_helper import *

In [ ]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import UpSampling2D
from keras.layers import Concatenate
from keras.layers import Lambda 
from keras.utils import to_categorical
import tensorflow as tf

from keras.layers import Reshape

from keras import backend as K
from keras import regularizers, optimizers
# %matplotlib inline

In [ ]:
def get_info(filenames, root, ext):
    images = []
    for filename in filenames :
        filepath = os.path.join(root,filename)
        if ext == '.npy':
            image = np.load(filepath)
            h,w = image.shape
            if h != 512 or w != 64:
                amount = 512 - h
                id_full = np.full((amount, 64), 0)
                try:
                    image = np.concatenate((image, id_full))
                except Exception as e:
                    print(image.shape)
        elif ext == '.JPG' or ext == '.tif' or ext =='.png':
            image = ndimage.imread(filepath)
        images.append(image)
    return images

def get_data(directory,ext):
    from os import listdir
    from os.path import isfile, join
    
    root_path = ""
    filenames = [f for f in listdir(directory) if isfile(join(directory, f)) and f != '.DS_Store']
    filenames = sorted(filenames)
    return filenames, get_info(filenames, directory, ext)

### Folder For Training Files

In [ ]:
# Setting the directories
import os

wanted_folder = 'alldata/'
# wanted_folder = 'pruned/'
# wanted_folder = 'Atrium/'
# wanted_folder = 'Ventricle/'

cwd = os.getcwd()
check_directory = cwd
if check_directory == '/home/sim/notebooks/relaynet_pytorch':
    cwd = cwd + '/datasets/OCTData/'+wanted_folder
elif check_directory == '/Users/sim/Desktop/Imperial/Project/PreTrained/relaynet_pytorch':
    cwd = cwd + '/datasets-24-aug/OCTData/'+wanted_folder

print(cwd)

#### Raw Files

In [ ]:
whole_raw_image_folder = cwd + 'whole_raw_image/'
print(whole_raw_image_folder)

In [ ]:
filenames, raw_images = get_data(whole_raw_image_folder, '.tif')

In [ ]:
print(len(filenames))
print (len(raw_images))
label = raw_images[0]
label[440:,:] = 0
plt.imshow(raw_images[0], cmap='gray')

### Labels

In [ ]:
manual_label_folder = cwd + 'manual_label/'
_, manual_labels = get_data(manual_label_folder,'.JPG')
print(filenames[0])
print(manual_labels[0].dtype)

In [ ]:
f, axs = plt.subplots(1,2,figsize=(20,20))
plt.subplot(121), plt.imshow(raw_images[0], cmap = "gray")
plt.title('Raw OCT Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(manual_labels[0])
plt.title('Manually Labelled Image'), plt.xticks([]), plt.yticks([])
# plt.subplot(133),plt.imshow(output)
# plt.title('Automated Label'), plt.xticks([]), plt.yticks([])
plt.show()

### Ids

In [ ]:
ids_folder = cwd + 'png_labels_method/'
_, ids = get_data(ids_folder,'.png')

print (len(ids))
print(ids[0].dtype)
plt.imshow(ids[0])

In [ ]:
h,w = 512, 600
data_shape = h*w
weight_decay = 0.0001
# Defines the input tensor
inputs = Input(shape=(h,w,1))

L1 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(inputs)
L2 = BatchNormalization()(L1)
L2 = Activation('relu')(L2)
#L3 = Lambda(maxpool_1,output_shape = shape)(L2)
L3 = MaxPooling2D(pool_size=(2,2))(L2)
L4 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L3)
L5 = BatchNormalization()(L4)
L5 = Activation('relu')(L5)
#L6 = Lambda(maxpool_2,output_shape = shape)(L5)
L6 = MaxPooling2D(pool_size=(2,2))(L5)
L7 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L6)
L8 = BatchNormalization()(L7)
L8 = Activation('relu')(L8)
#L9 = Lambda(maxpool_3,output_shape = shape)(L8)
L9 = MaxPooling2D(pool_size=(2,2))(L8)
L10 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L9)
L11 = BatchNormalization()(L10)
L11 = Activation('relu')(L11)
L12 = UpSampling2D(size = (2,2))(L11)
#L12 = Lambda(unpool_3,output_shape = unpool_shape)(L11)
L13 = Concatenate(axis = 3)([L8,L12])
L14 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L13)
L15 = BatchNormalization()(L14)
L15 = Activation('relu')(L15)
L16 = UpSampling2D(size= (2,2))(L15)
#L16 = Lambda(unpool_2,output_shape=unpool_shape)(L15)
L17 = Concatenate(axis = 3)([L16,L5])
L18 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L17)
L19 = BatchNormalization()(L18)
L19 = Activation('relu')(L19)
#L20 = Lambda(unpool_1,output_shape=unpool_shape)(L19)
L20 = UpSampling2D(size=(2,2),name = "Layer19")(L19)
L21 = Concatenate(axis=3)([L20,L2])
L22 = Conv2D(64,kernel_size=(3,3),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L21)
L23 = BatchNormalization()(L22)
L23 = Activation('relu')(L23)
L24 = Conv2D(8,kernel_size=(1,1),padding = "same",kernel_regularizer=regularizers.l2(weight_decay))(L23)
L = Reshape((data_shape,8),input_shape = (h,w,8))(L24)
L = Activation('softmax')(L)
model = Model(inputs = inputs, outputs = L)
# model.summary()

In [ ]:
def dice_coef(y_true, y_pred):
    '''
    y_true = label
    y_pred = prediction
    '''
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [ ]:
def customized_loss(y_true,y_pred):
    cross_ent = K.categorical_crossentropy(y_true, y_pred)
    loss_dice_coef = dice_coef_loss(y_true, y_pred)
    return (1 * cross_ent)+(0.5*loss_dice_coef)

In [ ]:
smooth = 1

In [ ]:
lrs = [0.01]
bs = 40
epoch = 100
for i in lrs:
    optimiser = optimizers.Adam(lr = i)
    model.compile(optimizer=optimiser,loss=customized_loss,metrics=['accuracy',dice_coef],sample_weight_mode='temporal')

In [ ]:
# model.load_weights("/home/sim/notebooks/relaynet_pytorch/"+saved_name+".hdf5")
# saved_name = 'notnormalised_bs50_ep_500_01'
saved_name = 'weightingval1_normalised_atriumventriclesubract_bs40_ep_200'
model.load_weights(check_directory+"/models/Normalised/"+saved_name+".hdf5")

In [ ]:
(15*600)/60

In [ ]:
# def one_hot_encode(inp, num_classes):
#     h,w = inp.shape
#     encoding = np.zeros(( h, w, num_classes))
#     for i in range(num_classes):
#         encoding[:,:,i] = inp == i
#     return encoding

# def list_of_labels(label_img, num_classes):

#     print(label_img.shape)
#     train_labels = one_hot_encode(label_img, num_classes)
#     return train_labels

In [ ]:
overall_stats = []
# for i in range(len(filenames)):
for i in range(2):
    ind = i

    # Raw Test Image 
    testing_image = raw_images[ind]
    test_label = manual_labels[ind]
    true_id = ids[ind]
    
    h,w = testing_image.shape

    testing_image = testing_image.reshape((1,h,w,1))
    prediction = model.predict(testing_image)
    prediction = np.squeeze(prediction,axis = 0)

    prediction = np.reshape(prediction,(h,w,8))

    print(prediction.shape)
    predicted_id = np.zeros((h,w))
    # ground = np.zeros((h,w))

    for i in range(h):
        for j in range(w):
            index = np.argmax(prediction[i][j]) # doing pixel wise prediction
            predicted_id[i][j] = index
#     idx = np.asarray(ids[0])
#     print(idx.shape)

    # Just used to plot the colours
    print(np.unique(predicted_id))
    
    # Creating one hot encoding of true labels and predicted labels
    true_labels = list_of_labels(true_id,8)
    pred_labels = list_of_labels(predicted_id,8)
    
    stats = find_stats(true_labels, pred_labels)
    overall_stats.append(stats)
    
    # Plotting Labels of the layers
#     fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(20,20))
#     for i, ax in enumerate(axes):
#         ax.imshow((true_labels[:,:,i]), alpha=0.2)
#         ax.set_title("label " + str(i))

#     fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(20,20))
#     for i, ax in enumerate(axes):
#         ax.imshow((pred_labels[:,:,i]), alpha=0.2)
#         ax.set_title("label " + str(i))
    
    
    
    # color = label_img_to_rgb(predicted_id) 
    
    
#     f, axs = plt.subplots(1,3,figsize=(20,20))
#     plt.subplot(131), plt.imshow(raw_images[ind], cmap = "gray")
#     plt.title('Raw OCT Image'), plt.xticks([]), plt.yticks([])
#     plt.subplot(132), plt.imshow(manual_labels[ind])
#     plt.title('Manually Labelled Image'), plt.xticks([]), plt.yticks([])
#     plt.subplot(133),plt.imshow(color)
#     plt.title('Automated Label'), plt.xticks([]), plt.yticks([])
#     plt.show()


# fig, axes = plt.subplots(nrows=8, ncols=1, figsize=(20,20))
# for i, ax in enumerate(axes):
#     ax.imshow(idx[:,:,i])
#     ax.set_title("slice " + str(i))
# plt.show()


In [ ]:
# for i in range(len(overall_stats)):
#     print('Image: {}, Stats: {}'.format(i,overall_stats[i]))
#     TP,FP,TN,FN,Acc, Precision, Recall, Dice = overall_stats[i]
#     print('Image:',i)
#     print('TP: {}, FP: {}, TN: {}, FN: {}, Class Accuracy: {}, Precision: {}, Recall: {}, Dice: {}'.format(TP,FP,TN,FN,Acc, Precision, Recall, Dice))
#     print()

In [ ]:
f, axs = plt.subplots(1,3,figsize=(20,20))
plt.subplot(131), plt.imshow(raw_images[ind], cmap = "gray")
plt.title('Raw OCT Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132), plt.imshow(manual_labels[ind])
plt.title('Manually Labelled Image'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(color)
plt.title('Automated Label'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
# for i in range(15,16):
ind = 0

# Raw Test Image 
testing_image = raw_images[ind]
test_labels = manual_labels[ind]
testing_image = testing_image[:,:]
# segmented_images = segment_image(testing_image, 0, 600, 64)

# testing_image = segmented_images[0]
h,w = testing_image.shape

testing_image = testing_image.reshape((1,h,w,1))
prediction = model.predict(testing_image)
prediction = np.squeeze(prediction,axis = 0)

prediction = np.reshape(prediction,(h,w,8))

print(prediction.shape)
output = np.zeros((h,w))
ground = np.zeros((h,w))
for i in range(h):
    for j in range(w):
        index = np.argmax(prediction[i][j])
        output[i][j] = index
idx = np.asarray(ids[0])
print(idx.shape)
color = label_img_to_rgb(output)